In [0]:
i = input("Please provide the input data : ")

def count_words(inputstr):
    strlist = inputstr.lower().split(" ")
    d = dict()
    for i in strlist:
        if i in d:
            d[i] += 1
        else:
            d[i] = 1
    return d

count_words(i)






In [0]:
from pyspark.sql.functions import explode,split






In [0]:
%sql
create volume if not exists streamvolume;

In [0]:
dbutils.fs.mkdirs("/Volumes/workspace/default/streamvolume/inputdata")

In [0]:
display(dbutils.fs.ls("/Volumes/workspace/default/streamvolume/"))

In [0]:

from pyspark.sql.functions import explode,split


df = spark.readStream.format("csv").option("header", "true").schema("value string").load("/Volumes/workspace/default/streamvolume/inputdata")

word_df = df.select(
    explode(split("value"," ")).alias("word")
    )

word_count_df = word_df.groupBy("word").count()

query = word_count_df.writeStream.outputMode("complete").format("memory").queryName("words_count").trigger(availableNow=True).option("checkpointLocation","/Volumes/workspace/default/streamvolume/checkpoint").start()

query.awaitTermination()
#display(workd_count_df, checkpointLocation = "/Volumes/workspace/default/streamvolume/checkpoint")




In [0]:
spark.sql("select * from words_count").show()